In [4]:
!ipc/ipc-static

ipc-static [-Bqsv] [-b buffersize] [-i pipe|local|tcp] [-p tcp_port]
	[-P l1d|l1i|l2|mem|tlb|axi] [-t totalsize] mode

Modes (pick one - default 1thread):
    1thread                IPC within a single thread
    2thread                IPC between two threads in one process
    2proc                  IPC between two threads in two different processes

Optional flags:
    -B                     Run in bare mode: no preparatory activities
    -i pipe|local|tcp      Select pipe, local sockets, or TCP (default: pipe)
    -p tcp_port            Set TCP port number (default: 10141)
    -P l1d|l1i|l2|mem|tlb|axi  Enable hardware performance counters
    -q                     Just run the benchmark, don't print stuff out
    -s                     Set send/receive socket-buffer sizes to buffersize
    -v                     Provide a verbose benchmark description
    -b buffersize          Specify a buffer size (default: 131072)
    -t totalsize           Specify total I/O size (default: 1677

In [5]:
!ipc/ipc-static -i pipe 2thread
!ipc/ipc-static -i local 2thread

274948.53 KBytes/sec
135210.06 KBytes/sec


In [6]:
!ipc/ipc-static -i local -s -b 1024 2thread

46141.98 KBytes/sec


In [7]:
!ipc/ipc-static -v -i pipe 2thread

Benchmark configuration:
  buffersize: 131072
  totalsize: 16777216
  blockcount: 128
  mode: 2thread
  ipctype: pipe
  time: 0.059394292
275851.42 KBytes/sec


In [8]:
!ipc/ipc-static -q -B -i pipe 2thread

In [14]:

setup_kernel()

kern.ipc.maxsockbuf: 33554432 -> 33554432


## Things to do
- [ ] instrument clock_gettime for the IPC loop
- [ ] it may be sensible to inspect quantize() results for both the execution time distributions of the system calls, and the amount of data returned by each
- [ ] investigate scheduling events using the sched provider ( on-cpu and off-cpu events)
- [ ] instrument sleep and wakeup
- [ ] take care about pid and tid
- [ ] probe effect: one simple way to approach the problem is to analyse the results of performance benchmarking with and without DTrace scripts running
- [ ] discard the first result
- [ ] read the FreeBSD Benchmarking Advice wiki
### questions to answer
- How does increasing IPC buffer size uniformly change performance across IPC models – and why?
- Explore the impact of the probe effect on your causal investigation; how has DTrace changed the behavior of the benchmark?


In [16]:
from collections import defaultdict


def setup_kernel():
    !sysctl kern.ipc.maxsockbuf=33554432
# Callback invoked to print the trace record
# (that is, printf("%u", vtimestamp - self->start))

def benchmark(flags, output_name, trials, buff_sizes, dtrace_script,quiet=False):
    values = []
    aggr_dict = defaultdict(int)
    
    def simple_out(value):
        values.append(value)
    
    def walk_func(a, b, c, d):
        aggr_dict[c[0]] += d
    
    # print_header(["Starting io-static performance measurement",flags])

        # Create a seperate thread to run the DTrace instrumentation
    setup_kernel()
    dtrace_thread = DTraceConsumerThread(dtrace_script,
                                    chew_func=lambda v: None,
                                        chewrec_func=lambda v: None,
                                        out_func=simple_out,
                                        walk_func=walk_func,
                                        sleep=1)

    # Start the DTrace instrumentation
    dtrace_thread.start()

    # Display header to indicate that the benchmarking has started
    for size in buff_sizes:
        if not quiet:
            print("buffer size:", size, "collected so far: ",len(values))
        for i in range(trials):
            output = !ipc/ipc-static -B -q -b {str(size)} {flags}
            
    # The benchmark has completed - stop the DTrace instrumentation
    dtrace_thread.stop()
    dtrace_thread.join()
    dtrace_thread.consumer.__del__()

    if not quiet:
        print("values collected:", len(values))

    with open("lab1_{}_0702.data".format(output_name), 'w') as f:
        f.write(",".join([str(s) for s in BUFFER_SIZES]))
        f.write("\n")
        for value in values:
            f.write(value)
            f.write("\n")
    return values, aggr_dict

In [ ]:
# IO performance varying the buffer size
io_performance_script = """

BEGIN {
}

syscall::clock_gettime:return
/execname == "io-static" && !self->in_benchmark/
{
    self->in_benchmark = 1;
    self->cstart = vtimestamp;
}

syscall::clock_gettime:entry
/execname == "io-static" && self->in_benchmark/
{
    self->in_benchmark = 0;
    trace(vtimestamp - self->cstart);
}

END
{
    exit(0);
}
"""
